In [11]:
import os
import numpy as np
from scipy.spatial.distance import cdist

def parse_file(file_path, n=20):
    instances = []
    with open(file_path, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if "output" in tokens:
                idx = tokens.index("output")
                tokens = tokens[:idx]  # output 이전까지만 자르기
            try:
                coords_flat = list(map(float, tokens))
                if len(coords_flat) == 2 * n:
                    coords = np.array(coords_flat).reshape((n, 2))
                    instances.append(coords)
            except ValueError:
                continue
    return instances

def parse_file_with_optimal_tour(file_path, n=20):
    instances = []
    with open(file_path, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if "output" in tokens:
                idx = tokens.index("output")
                coords_flat = list(map(float, tokens[:idx]))
                tour = [int(x) - 1 for x in tokens[idx + 1:]]
                if len(coords_flat) == 2 * n and len(tour) == n + 1:
                    coords = np.array(coords_flat).reshape((n, 2))
                    instances.append((coords, tour))
    return instances

def save_distance_matrix_with_tour_txt(coords, tour, file_path):
    n = coords.shape[0]
    dist_matrix = cdist(coords, coords)
    dist_values = []
    for i in range(n):
        for j in range(i + 1, n):
            dist_values.append(dist_matrix[i, j])
    with open(file_path, 'w') as f:
        for val in dist_values:
            f.write(f"{val} ")
        f.write("output ")
        for node in tour:
            f.write(f"{node + 1} ")  # 다시 1-indexed로 저장
        f.write("\n")

input_path = os.path.join("dataset", "hw_2_tsp_20.txt")
instances_with_tour = parse_file_with_optimal_tour(input_path)

for i, (coords, tour) in enumerate(instances_with_tour, 1):
    file_path = os.path.join("dataset", f"20-{i}.txt")
    save_distance_matrix_with_tour_txt(coords, tour, file_path)

In [ ]:
import os
import time
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming

# 설정
node_counts = [10, 15, 20]
num_samples = 500
seed = 1234

np.random.seed(seed)

for n in node_counts:
    # 폴더 준비
    base_dir = f"data/TSP{n}"
    inst_dir = os.path.join(base_dir, "instances")
    sol_dir  = os.path.join(base_dir, "solutions")
    os.makedirs(inst_dir, exist_ok=True)
    os.makedirs(sol_dir,  exist_ok=True)

    start_all = time.time()
    valid = 0

    while valid < num_samples:
        # 1) 무작위 좌표 생성
        coords = np.random.random((n, 2))  # [0,1]^2

        # 2) 거리 행렬 계산
        dist_matrix = np.linalg.norm(
            coords[:, None, :] - coords[None, :, :],
            axis=2
        )

        # 3) DP 기반 exact TSP 해 계산
        t0 = time.time()
        try:
            tour, cost = solve_tsp_dynamic_programming(dist_matrix)
        except MemoryError:
            # n이 너무 크면 DP가 메모리 부족할 수 있음
            continue
        elapsed = time.time() - t0

        # 4) 정상 투어인 경우 저장
        if tour is not None and len(tour) == n:
            fname = f"{valid:03d}"

            # 좌표 저장
            np.savez_compressed(
                os.path.join(inst_dir, f"instance_{fname}.npz"),
                coords=coords
            )

            # 솔루션(투어, 코스트, 소요 시간) 저장
            np.savez_compressed(
                os.path.join(sol_dir, f"solution_{fname}.npz"),
                tour=tour,
                cost=cost,
                solve_time_s=elapsed
            )

            valid += 1
            print(f"[TSP{n} #{fname}] cost={cost:.2f}, time={elapsed:.2f}s")

#    print(f"TSP{n}: Generated {valid} instances in {time.time() - start_all:.1f}s")